# Image Classification using Convolutional Neural Network

In [1]:
# Importing necessary Libraries
import pandas as pd
import numpy  as np
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [2]:
# Rotating, Turning and Flipping our images 
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip=True)

In [3]:
# Loading and Pre-processing the dataset

test_datagen = ImageDataGenerator(rescale = 1./255)

# Creating the Training dataset
training_set = train_datagen.flow_from_directory("cat_and_dog_datasets/training_set/training_set",target_size = (64,64),batch_size = 32,class_mode = "binary",color_mode="grayscale")

# Creating the Testing dataset
testing_set = test_datagen.flow_from_directory("cat_and_dog_datasets/test_set/test_set",target_size = (64,64),batch_size = 32,class_mode = "binary",color_mode="grayscale")

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [4]:
# Building the Convolutinal Neural Network
classifier = Sequential()
classifier.add(Convolution2D(32,3,3,input_shape=(64,64,1),activation="relu"))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(128,activation="relu"))
classifier.add(Dense(1,activation="sigmoid"))

In [5]:
# Compiling the CNN Model
classifier.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [6]:
# Training the CNN Model
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor="val_loss",mode="min",patience=5,restore_best_weights=True,verbose = 1)
classifier.fit(training_set,steps_per_epoch=100,epochs=10,validation_data=testing_set,validation_steps=25,callbacks=[earlystopping])

Epoch 1/10
100/100 [==============================] - 19s 182ms/step - loss: 0.6936 - accuracy: 0.5203 - val_loss: 0.6910 - val_accuracy: 0.4900
Epoch 2/10
100/100 [==============================] - 19s 187ms/step - loss: 0.6751 - accuracy: 0.5816 - val_loss: 0.6680 - val_accuracy: 0.5450
Epoch 3/10
100/100 [==============================] - 19s 186ms/step - loss: 0.6558 - accuracy: 0.6136 - val_loss: 0.6303 - val_accuracy: 0.6562
Epoch 4/10
100/100 [==============================] - 18s 181ms/step - loss: 0.6382 - accuracy: 0.6375 - val_loss: 0.6301 - val_accuracy: 0.6400
Epoch 5/10
100/100 [==============================] - 18s 176ms/step - loss: 0.6252 - accuracy: 0.6511 - val_loss: 0.6216 - val_accuracy: 0.6550
Epoch 6/10
100/100 [==============================] - 19s 189ms/step - loss: 0.6073 - accuracy: 0.6644 - val_loss: 0.6296 - val_accuracy: 0.6488
Epoch 7/10
100/100 [==============================] - 18s 178ms/step - loss: 0.5999 - accuracy: 0.6719 - val_loss: 0.6402 - val_ac

In [13]:
# Classifying the New Images using the Trained CNN Model
import tensorflow as tf

def predict_image(imagepath):
    
    path    = imagepath
    predict = tf.keras.preprocessing.image.load_img(path, target_size=(64,64),color_mode="grayscale")
    predict_modified = tf.keras.preprocessing.image.img_to_array(predict)
    predict_modified = predict_modified / 255
    predict_modified = np.expand_dims(predict_modified,axis=0)
    
    result = classifier.predict(predict_modified)
    
    if result[0][0] > 0.5:
        prediction = "dog"
        probability = result[0][0]
    else:
        prediction = "cat"
        probability = 1 - result[0][0]
    print(prediction)
    print("probability = "+str(probability))

predict_image("cat_and_dog_datasets/new_images/cat.1.jpg")
predict_image("cat_and_dog_datasets/new_images/dog.4001.jpg")        

1/1 [==============================] - 0s 22ms/step
cat
probability = 0.508996844291687
1/1 [==============================] - 0s 21ms/step
dog
probability = 0.7277723
